# Machine Learning Coarse Graining

## Overview:

This tutorial demonstrates how to use a pre-trained machine learning model to predict forces in a coarse-grained system.

In [1]:
from flowermd import Simulation

### Isotropic Coarse Graining with FlowerMD

We first load a pre-trained pytorch model and apply it to system of spherical (isotropic) particles.
This model uses neighbor lists to predict forces between particles in the system. The model is trained on a dataset of Lennard-Jones systems with 200 particles.

We use a pre-trained pytorch neural network to determine the forces between particles based on the relative distance vector between each particle and its 40th nearest neighbors. This model was trained for 1000 epochs with a batch size of 128 and an Adam optimizer. The best model checkpoint is saved to a file, which we will load and apply to a simulation.

The `load_isotropic_custom_force` function loads the pre-trained model and creates a custom force object that can be applied to a simulation. This custom force object is like a look-up function for the simulation, providing the net force on each particle at each time step.

The `load_isotropic_custom_force` function loads the pre-trained model and creates a custom force object that can be applied to a simulation. This custom force object is like a look-up function for the simulation, providing the net force on each particle at each time step.

In [ ]:
from flowermd.modules.coarse_graining_ML.isotropic.isotropic_coarse_graining import (
    load_isotropic_custom_force,
    LJNeighborModelConfig,
)

best_model_path = "data/isotropic_best_model_checkpoint.pth"
lj_system = "data/LJ_system_N200.gsd"
model_config = LJNeighborModelConfig(
    number_neighbors=40,
    hidden_dim=128,
    n_layers=3,
    act_fn="Tanh",
    dropout=0.3,
    batch_norm=False,
    box_len=6,
    prior_energy=True,
    prior_energy_sigma=1,
    prior_energy_n=12,
)
custom_force = load_isotropic_custom_force(
    model_config, best_model_path=best_model_path
)
sim = Simulation(
    initial_state=lj_system,
    forcefield=[custom_force],
    gsd_file_name="LJ_trajectory.gsd",
    log_file_name="LJ_log.txt",
)
sim.run_NVT(temperature=3.0, duration=10000, tau_kt=0.1)
sim.flush_writers()
print("done")

#

### Anisotropic Coarse Graining with FlowerMD

This example demonstrates how to use a pre-trained machine learning model to predict forces and torques in a coarse-grained system of anisotropic particles. The model is trained on a dataset of anisotropic system with 200 PPS monomers. Two models are trained: one for predicting forces and another for predicting torques. The model uses neighbor lists to predict forces and torques between particles in the system.

In [6]:
from flowermd.modules.coarse_graining_ML.anisotropic import (
    load_anisotropic_custom_force,
    ForTorPredictorNNModelConfig,
)
import unyt as u

force_model_path = "data/aniso_force_model_checkpoint.pth"
torque_model_path = "data/aniso_torque_model_checkpoint.pth"
pps_system = "data/PPS_system_N200.gsd"
model_config = ForTorPredictorNNModelConfig(
    number_neighbors=30,
    hidden_dim=512,
    n_layers=3,
    act_fn="Tanh",
    dropout=0.3,
    batch_norm=False,
    box_len=9.551452,
)
custom_force = load_anisotropic_custom_force(
    model_config,
    force_model_path=force_model_path,
    torque_model_path=torque_model_path,
)
ref_values = {
    "energy": 1.7782 * u.Unit("kJ/mol"),
    "length": 0.36 * u.Unit("nm"),
    "mass": 32.06 * u.Unit("amu"),
}
sim = Simulation(
    initial_state=pps_system,
    forcefield=[custom_force],
    gsd_file_name="PPS_aniso_trajectory.gsd",
    log_file_name="PPS_aniso_log.txt",
    integrate_rotational_dof=True,
    reference_values=ref_values,
)
sim.run_NVT(temperature=3.0, duration=1000, tau_kt=0.1)
sim.flush_writers()
print("done")

Initializing simulation state from a GSD file.
done
